In [1]:
import numpy as np
import pandas as pd
import os
from os.path import join
from os.path import exists

import json

In [2]:
dataDir = '../data'

# Process Collaborations
Take the processed articles and create a table of collaborations across all articles

In [3]:
# read processed articles
arts = []
with open(join(dataDir, 'articleMetadata.json')) as f:
    for i,line in enumerate(f):
        thisArt = json.loads(line)
        if 'isValid' in thisArt.keys():
            arts.append(thisArt)

## Filter article list
Only include articles with at least one cross-institutional collaboration, defined as two or more unique locations among the author affiliations

In [4]:
arts[5]

{'PMCID': 'PMC7568024',
 'isValid': True,
 'title': 'COVID-19-Komplikationen nach bariatrischer OP erhöht bei Diabetes wie niedrigem BMI',
 'authors': {'authors': [{'surname': 'Müssig',
    'givenNames': 'Karsten',
    'aff': 'Aff1'}],
  'affiliations': [{'id': 'Aff1',
    'address': 'Franziskus-Hospital Harderberg, Alte Rothenfelder Str. 23, 49124 Georgsmarienhütte, Deutschland',
    'locID': 'loc000016514',
    'hasGeo': True,
    'geoID': 'ChIJLzKVYEjuuUcRsJQAY3fPJgQ'}]},
 'pubDates': [{'pubType': 'epub', 'date': '10-27-2020'},
  {'pubType': 'ppub', 'date': '01-01-2020'}],
 'pmid': None,
 'doi': '10.1007/s15034-020-2195-x',
 'journalTitle': 'Info Diabetologie',
 'refs': [{'type': 'doi', 'id': '10.1002/dmrr.3319'}],
 'URL': 'https://www.ncbi.nlm.nih.gov/pmc/oai/oai.cgi?verb=GetRecord&identifier=oai:pubmedcentral.nih.gov:7568024&metadataPrefix=pmc',
 'nAuthors': 1,
 'nLocations': 1,
 'nLocsWithGeo': 1,
 'pubDate': '2020-10-27'}

In [5]:
def getUniqueGeos(art):
    # return the number of unique geoIDs (i.e. cities) associated with this article
    geoIDs = []
    for aff in art['authors']['affiliations']:
        if aff['hasGeo']:
            if aff['geoID'] not in geoIDs:
                geoIDs.append(aff['geoID'])
                
    return geoIDs

In [6]:
filteredArts = [art for art in arts if len(getUniqueGeos(art)) >= 2]

In [7]:
print('{} total articles'.format(len(arts)))
print('{} articles with 2 or more unique geoIDs'.format(len(filteredArts)))

93593 total articles
34218 articles with 2 or more unique geoIDs


## Create dataframe of collaborations
Each row will represent one unique cross-geo collaboration. A cross-geo collaboration is defined as any 2 investigators from affiliations with unique geo IDs co-authoring a given article

*Example*  
**A study investigating a really important question** (2020) *Journal of Wow*  
James Osterberg - Wayne State, Detroit, MI  
Ron Asheton - University of Michigan, Ann Arbor, MI  
James Williamson - NYU, New York City, NY  

This article would comprise 3 cross-geo collaborations:
* Detroit - Ann Arbor
* Ann Arbor - New York
* New York - Detroit




In [8]:
import random

In [9]:
from itertools import combinations

In [10]:
pubDates_df = pd.read_csv(join(dataDir, 'processed', 'publicationDates.csv'))

In [11]:
pubDates_df.head()

,PMCID,pubDate
0,PMC7250740,2020-05-27
1,PMC7573915,2020-09-18
2,PMC7454712,2020-08-12
3,PMC7439822,2020-08-20
4,PMC7530335,2020-09-18


In [12]:
def getCollaborations(art):
    # return a dataframe of the unique cross geo collaborations for this article
    geoIDs = getUniqueGeos(art);
    geoIDs.sort()
    
    # get unique pairs
    geoPairs = {'geoID_A': [], 'geoID_B': []}
    pairs = combinations(geoIDs, 2)

    for i in list(pairs):
        geoPairs['geoID_A'].append(i[0])
        geoPairs['geoID_B'].append(i[1])
    
    df = pd.DataFrame(geoPairs)
    df['PMCID'] = art['PMCID']
    df['pubDate'] = art['pubDate']
    return df[['PMCID', 'pubDate', 'geoID_A', 'geoID_B']]
    
    

This step takes ~3-5 mins to run

In [13]:
collabs_df = pd.DataFrame(columns=['PMCID', 'pubDate', 'geoID_A', 'geoID_B'])

for art in filteredArts:
    this_df = getCollaborations(art)
    
    collabs_df = pd.concat([collabs_df, this_df], ignore_index=True)

In [14]:
collabs_df.shape

(367423, 4)

In [15]:
collabs_df.head()

,PMCID,pubDate,geoID_A,geoID_B
0,PMC7581440,2020-10-23,ChIJ7faBuL7or0cRYDuslG2sJQQ,ChIJAVkDPzdOqEcRcDteW0YgIQQ
1,PMC7581502,2020-10-23,ChIJc8r44c9unUcRDZsdKH0cIJ0,ChIJuRMYfoNhsUcRoDrWe_I9JgQ
2,PMC7581692,2020-10-23,ChIJmb1k2ko-eUgRqdwTAv26rVE,ChIJra6o8IHuBUgRMO0NHlI3DQQ
3,PMC7581692,2020-10-23,ChIJmb1k2ko-eUgRqdwTAv26rVE,ChIJsZ3dJQevthIRAuiUKHRWh60
4,PMC7581692,2020-10-23,ChIJmb1k2ko-eUgRqdwTAv26rVE,ChIJt2BwZIrfekgRAW4XP28E3EI


# Add location info to the geoIDs of each collaboration

In [16]:
geo_df = pd.read_csv(join(dataDir, 'cityGeocodes.csv'))

In [17]:
def getLocationInfo(geoID):
    """ returns a series with 3 values - lat, lng, formattedAddr"""
    match = geo_df.loc[geo_df['geoID'] == geoID]

    return pd.Series({
            'lat': match['lat'].item(),
            'lng': match['lng'].item(),
            'fmtAddr': match['formattedAddr'].item()
        })


This step takes ~25-30 min to run

In [18]:
print('getting location info for A column')
collabs_df[['lat_A', 'lng_A', 'fmtAddr_A']] = collabs_df['geoID_A'].apply(getLocationInfo)

print('getting location info for B column')
collabs_df[['lat_B', 'lng_B', 'fmtAddr_B']] = collabs_df['geoID_B'].apply(getLocationInfo)

print('done')

getting location info for A column
getting location info for B column
done


In [19]:
collabs_df.head()

,PMCID,pubDate,geoID_A,geoID_B,lat_A,lng_A,fmtAddr_A,lat_B,lng_B,fmtAddr_B
0,PMC7581440,2020-10-23,ChIJ7faBuL7or0cRYDuslG2sJQQ,ChIJAVkDPzdOqEcRcDteW0YgIQQ,52.480909,10.550783,"38518 Gifhorn, Germany",52.520007,13.404954,"Berlin, Germany"
1,PMC7581502,2020-10-23,ChIJc8r44c9unUcRDZsdKH0cIJ0,ChIJuRMYfoNhsUcRoDrWe_I9JgQ,47.269212,11.404102,"Innsbruck, Austria",53.551085,9.993682,"Hamburg, Germany"
2,PMC7581692,2020-10-23,ChIJmb1k2ko-eUgRqdwTAv26rVE,ChIJra6o8IHuBUgRMO0NHlI3DQQ,53.800755,-1.549077,"Leeds, UK",47.218371,-1.553621,"Nantes, France"
3,PMC7581692,2020-10-23,ChIJmb1k2ko-eUgRqdwTAv26rVE,ChIJsZ3dJQevthIRAuiUKHRWh60,53.800755,-1.549077,"Leeds, UK",43.610769,3.876716,"Montpellier, France"
4,PMC7581692,2020-10-23,ChIJmb1k2ko-eUgRqdwTAv26rVE,ChIJt2BwZIrfekgRAW4XP28E3EI,53.800755,-1.549077,"Leeds, UK",53.408371,-2.991573,"Liverpool, UK"


### Save Collaborations dataframe

In [20]:
collabs_df.to_csv(join(dataDir, 'processed', 'collaborations.csv'), index=False)

# Process GEO IDs 
Using the collaborations dataframe, create a new table with one row for each GEOID and columns indicating the total number of collabs and the number of unique collabs

In [21]:
geo_df.shape

(8044, 7)

This step takes ~10 mins to run

In [22]:
collabsByGeo = []
for geoID in geo_df['geoID'].values.tolist():
    # get all collabs that include this id
    collabs = collabs_df.loc[(collabs_df['geoID_A'] == geoID) | (collabs_df['geoID_B'] == geoID)]
    
    # total number of collaborations this id took part in
    nTotal = collabs.shape[0]
    
    # get unique collaborations this id took part in
    uniqueIDs = set(collabs['geoID_A'].values.tolist() + collabs['geoID_B'].values.tolist())
    nUnique = len(uniqueIDs) - 1 # minus one to remove the current geoID
    if nUnique == -1:
        nUnique = 0
        
    # calculate a collaboration uniqueness ratio (nUnique / nTotal)
    if nTotal == 0:
        uniqueRatio = 0
    else:
        uniqueRatio = nUnique / nTotal
    
    # add this info to the list
    collabsByGeo.append({
        'geoID': geoID,
        'nTotal': nTotal,
        'nUnique': nUnique,
        'uniqueRatio': uniqueRatio
    })

# convert to dataframe
collabsByGeo_df = pd.DataFrame(collabsByGeo)

# get location info for each geoID
collabsByGeo_df[['lat', 'lng', 'formattedAddr']] = collabsByGeo_df['geoID'].apply(getLocationInfo)

In [23]:
collabsByGeo_df.head()

,geoID,nTotal,nUnique,uniqueRatio,lat,lng,formattedAddr
0,ChIJIQBpAG2ahYAR_6128GcTUEo,3123,862,0.276017,37.774929,-122.419415,"San Francisco, CA, USA"
1,ChIJtzQTOkE8JV4RZpGSzvDkmVQ,353,138,0.390935,41.805699,123.431472,"Shenyang, Liaoning, China"
2,ChIJ4WRJ3RLvf0cRKH8rSX0edZY,643,244,0.379471,44.647128,10.925227,"Modena, Province of Modena, Italy"
3,ChIJwbIYXknIlkcRHyTnGDFIGpc,957,290,0.303030,48.573405,7.752111,"Strasbourg, France"
4,ChIJdd4hrwug2EcRmSrV3Vo6llI,12497,1883,0.150676,51.507351,-0.127758,"London, UK"


### Save

In [24]:
collabsByGeo_df.to_csv(join(dataDir, 'processed', 'collabsByGeo.csv'), index=False)

# Misc

In [80]:
collabsByGeo_df.sort_values('nUnique', ascending=False).head(20)

,geoID,nTotal,nUnique,uniqueRatio,lat,lng,formattedAddr
4,ChIJdd4hrwug2EcRmSrV3Vo6llI,11655,1837,0.157615,51.507351,-0.127758,"London, UK"
17,ChIJOwg_06VPwokRYv534QaPC8g,8329,1597,0.191740,40.712775,-74.005973,"New York, NY, USA"
19,ChIJGzE9DS1l44kRoOhiASS_fHg,8453,1419,0.167869,42.360082,-71.058880,"Boston, MA, USA"
13,ChIJD7fiBh9u5kcRYJSMaMOCCwQ,6171,1233,0.199806,48.856614,2.352222,"Paris, France"
190,ChIJ53USP0nBhkcRjQ50xhPN_zw,5639,1203,0.213336,45.464204,9.189982,"Milan, Metropolitan City of Milan, Italy"
74,ChIJgTwKgJcpQg0RaSKMYcHeNsQ,5474,1165,0.212824,40.416775,-3.703790,"Madrid, Spain"
265,ChIJVTPokywQkFQRmtVEaUZlJRA,4317,1100,0.254807,47.606209,-122.332071,"Seattle, WA, USA"
233,ChIJ5TCOcRaYpBIRCmZHTz37sEQ,5287,1099,0.207868,41.385064,2.173404,"Barcelona, Spain"
116,ChIJuSwU55ZS8DURiqkPryBWYrk,4923,1088,0.221003,39.904200,116.407396,"Beijing, China"
95,ChIJpTvG15DL1IkRd8S0KlBVNTI,4561,1085,0.237886,43.653226,-79.383184,"Toronto, ON, Canada"


In [73]:
collabsByGeo_df.sort_values('nTotal', ascending=False).head(20)

,geoID,nTotal,nUnique,lat,lng,formattedAddr
4,ChIJdd4hrwug2EcRmSrV3Vo6llI,11655,1837,51.507351,-0.127758,"London, UK"
19,ChIJGzE9DS1l44kRoOhiASS_fHg,8453,1419,42.360082,-71.058880,"Boston, MA, USA"
17,ChIJOwg_06VPwokRYv534QaPC8g,8329,1597,40.712775,-74.005973,"New York, NY, USA"
13,ChIJD7fiBh9u5kcRYJSMaMOCCwQ,6171,1233,48.856614,2.352222,"Paris, France"
190,ChIJ53USP0nBhkcRjQ50xhPN_zw,5639,1203,45.464204,9.189982,"Milan, Metropolitan City of Milan, Italy"
74,ChIJgTwKgJcpQg0RaSKMYcHeNsQ,5474,1165,40.416775,-3.703790,"Madrid, Spain"
233,ChIJ5TCOcRaYpBIRCmZHTz37sEQ,5287,1099,41.385064,2.173404,"Barcelona, Spain"
56,ChIJu46S-ZZhLxMROG5lkwZ3D7k,5010,1073,41.902784,12.496365,"Rome, Metropolitan City of Rome, Italy"
116,ChIJuSwU55ZS8DURiqkPryBWYrk,4923,1088,39.904200,116.407396,"Beijing, China"
95,ChIJpTvG15DL1IkRd8S0KlBVNTI,4561,1085,43.653226,-79.383184,"Toronto, ON, Canada"
